In [1]:
import pandas as pd

df = pd.read_csv("yeobo.csv")

df["강수여부"] = df["일강수량(mm)"].apply(lambda x: 1 if pd.notna(x) and x > 0 else 0)

mean_cloud_with_rain = df.loc[df["강수여부"] == 1, "평균 전운량(1/10)"].mean()
mean_cloud_without_rain = df.loc[df["강수여부"] == 0, "평균 전운량(1/10)"].mean()

def fill_cloud(row):
    if pd.isna(row["평균 전운량(1/10)"]):
        return mean_cloud_with_rain if row["강수여부"] == 1 else mean_cloud_without_rain
    else:
        return row["평균 전운량(1/10)"]

df["평균 전운량(1/10)"] = df.apply(fill_cloud, axis=1)

df.to_csv("yeobo_processed.csv", index=False)


In [3]:
df = pd.read_csv("yeobo_processed.csv")
print(df['강수여부'])

0        1
1        0
2        0
3        0
4        1
        ..
14607    0
14608    1
14609    0
14610    0
14611    0
Name: 강수여부, Length: 14612, dtype: int64


In [7]:
import pandas as pd

# 1. 파일 불러오기
df = pd.read_csv("yeobo.csv")

# 2. 날짜 타입으로 변환 및 정렬
df["일시"] = pd.to_datetime(df["일시"])
df = df.sort_values("일시")
df.set_index("일시", inplace=True)

# 3. 강수여부 생성
df["강수여부"] = df["일강수량(mm)"].apply(lambda x: 1 if pd.notna(x) and x > 0 else 0)

# 4. 평균 전운량 결측치 채우기
mean_cloud_with_rain = df.loc[df["강수여부"] == 1, "평균 전운량(1/10)"].mean()
mean_cloud_without_rain = df.loc[df["강수여부"] == 0, "평균 전운량(1/10)"].mean()

df["평균 전운량(1/10)"] = df.apply(
    lambda row: mean_cloud_with_rain if pd.isna(row["평균 전운량(1/10)"]) and row["강수여부"] == 1
    else mean_cloud_without_rain if pd.isna(row["평균 전운량(1/10)"]) else row["평균 전운량(1/10)"],
    axis=1
)

# 5. 풍속 및 상대습도 결측 보완
# interpolate (시간 기준, limit=1)
for col in ["평균 풍속(m/s)", "평균 상대습도(%)"]:
    df[col] = df[col].interpolate(method="time", limit=1)

# 평균 풍속 보완 (월/일 기준 과거 평균)
df["월"] = df.index.month
df["일"] = df.index.day

for col in ["평균 풍속(m/s)"]:
    daily_avg = df.groupby(["월", "일"])[col].transform("mean")
    df[col] = df[col].fillna(daily_avg)

# 상대습도 보완 (전체 평균값)
for col in ["평균 상대습도(%)"]:
    df[col] = df[col].fillna(df[col].mean())

# 6. 정리 후 저장
df.drop(columns=["월", "일"], inplace=True)
df.reset_index(inplace=True)
df.to_csv("yeobo_processed.csv", index=False)


In [26]:
df = pd.read_csv("yeobo_processed.csv")
df.isna().sum()

일시                      0
지점                      0
지점명                     0
평균기온(°C)                7
일강수량(mm)             8034
평균 풍속(m/s)              0
평균 이슬점온도(°C)            7
평균 상대습도(%)              0
평균 증기압(hPa)             7
평균 현지기압(hPa)            8
평균 해면기압(hPa)            8
합계 일조시간(hr)            29
합계 일사량(MJ/m2)        7038
일 최심신적설(cm)         14525
일 최심적설(cm)          14515
합계 3시간 신적설(cm)      14525
평균 전운량(1/10)            0
평균 중하층운량(1/10)       3855
평균 지면온도(°C)            29
평균 5cm 지중온도(°C)     10973
평균 10cm 지중온도(°C)    10974
평균 20cm 지중온도(°C)    10980
평균 30cm 지중온도(°C)    10964
합계 대형증발량(mm)         7811
합계 소형증발량(mm)         7586
9-9강수(mm)           11216
안개 계속시간(hr)         14385
강수여부                    0
dtype: int64

In [38]:
df = pd.read_csv("yeobo.csv")
print(df.isna().sum())

지점                      0
지점명                     0
일시                      0
평균기온(°C)                7
일강수량(mm)             8034
평균 풍속(m/s)             44
평균 이슬점온도(°C)            7
평균 상대습도(%)              6
평균 증기압(hPa)             7
평균 현지기압(hPa)            8
평균 해면기압(hPa)            8
합계 일조시간(hr)            29
합계 일사량(MJ/m2)        7038
일 최심신적설(cm)         14525
일 최심적설(cm)          14515
합계 3시간 신적설(cm)      14525
평균 전운량(1/10)         3550
평균 중하층운량(1/10)       3855
평균 지면온도(°C)            29
평균 5cm 지중온도(°C)     10973
평균 10cm 지중온도(°C)    10974
평균 20cm 지중온도(°C)    10980
평균 30cm 지중온도(°C)    10964
합계 대형증발량(mm)         7811
합계 소형증발량(mm)         7586
9-9강수(mm)           11216
안개 계속시간(hr)         14385
dtype: int64


In [40]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer  # MICE 기능 활성화
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

# 결측치 0으로 처리할 열
fill_zero_columns = ['일강수량(mm)', '일 최심신적설(cm)']
for col in fill_zero_columns:
    null_count = df[col].isnull().sum()
    df[col] = df[col].fillna(0)
    print(f"[{col}] 결측치 {null_count}개를 0으로 채움.")


target_columns = [
    '평균기온(°C)', '평균 풍속(m/s)','평균 상대습도(%)','평균 전운량(1/10)','평균 지면온도(°C)', ']

imputer = IterativeImputer(
    estimator=RandomForestRegressor(n_estimators=100, random_state=42),
    max_iter=10,
    random_state=42
)

# 수치형 데이터만 선택 (특히 대상 열만 사용)
imputed_data = imputer.fit_transform(df[target_columns])

# 결과를 원래 데이터프레임에 반영
df[target_columns] = imputed_data

# 저장
df.to_csv("weather_all_data_filled_mice.csv", index=False, encoding="utf-8-sig")
print("MICE 방식(RandomForest 기반)으로 결측치 채우기 완료 및 저장됨")


[일강수량(mm)] 결측치 0개를 0으로 채움.
[일 최심신적설(cm)] 결측치 0개를 0으로 채움.
MICE 방식(RandomForest 기반)으로 결측치 채우기 완료 및 저장됨


C:\Users\MINJUN\Documents\GitHub\Data_Baisc_Analyze\Data_Baisc_Analyze\.venv\Lib\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [42]:
df = pd.read_csv("weather_all_data_filled.csv")
print(df.isna().sum())

지점                      0
지점명                     0
일시                      0
평균기온(°C)                0
일강수량(mm)                0
평균 풍속(m/s)              0
평균 이슬점온도(°C)            7
평균 상대습도(%)              0
평균 증기압(hPa)             7
평균 현지기압(hPa)            8
평균 해면기압(hPa)            8
합계 일조시간(hr)            29
합계 일사량(MJ/m2)        7038
일 최심신적설(cm)             0
일 최심적설(cm)          14515
합계 3시간 신적설(cm)      14525
평균 전운량(1/10)            0
평균 중하층운량(1/10)       3855
평균 지면온도(°C)             0
평균 5cm 지중온도(°C)     10973
평균 10cm 지중온도(°C)    10974
평균 20cm 지중온도(°C)    10980
평균 30cm 지중온도(°C)    10964
합계 대형증발량(mm)         7811
합계 소형증발량(mm)         7586
9-9강수(mm)           11216
안개 계속시간(hr)         14385
dtype: int64
